## Grundlagen Maschineller Lernverfahren | ML_INF19A | 2021
**Datum: 02.11.2021**

# Transfer Learning (Neuronale Netzwerke)
Ressourcen: https://www.tensorflow.org/api_docs/python/tf

## Lade Daten

Verwende CIFAR10 Datensatz

In [ ]:
# Lade Bibliotheken
from tensorflow.keras.datasets import cifar10 # https://www.cs.toronto.edu/~kriz/cifar.html

In [ ]:
# Lade den Datensatz
# Hinweis: Kann beim ersten Aufruf den Download starten
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

## Analysiere

In [ ]:
# Anzeige der Dimensionen
print(X_train.shape) # -> 50.000 32x32x3 Bilder (RGB)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
print(X_train[0])
print(y_train[0])

In [ ]:
# Blick in die Label
print(y_train) # -> nur Zahlen

In [ ]:
# Nutze Informationen von der Homepage des Datensatzes um den Integer-Werten der Klassen aussagekräftige Labelnamne zuzuweisen
# vgl. https://www.cs.toronto.edu/~kriz/cifar.html

labels = ['Flugzeug', 'Auto', 'Vogel', 'Katze', 'Wild', 'Hund', 'Frosch', 'Pferd', 'Schiff', 'LKW']

In [ ]:
# Lade Bibliotheken
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Gebe ein beliebiges Bild aus
rnd_image = np.random.randint(0, X_train.shape[0]) # Wähle zufällig einen Index

plt.imshow(X_train[rnd_image])
plt.xticks([])
plt.yticks([])
plt.xlabel("Bild #%d (%s)\n" % (rnd_image, labels[y_train[rnd_image][0]]))

## Daten aufbereiten

In [ ]:
# Lade Bibliothek zum Skalieren
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

# Multipliziere alle Werte mit 1/255
X_train = Rescaling(scale=1./255)(X_train)
X_test = Rescaling(scale=1./255)(X_test)
print(X_train) # Ausgabe ist jetzt Tensor (kein numpy Array mehr!)
#print(X_test)

In [ ]:
# Exkurs: Tensor in Numpyarray konvertieren
X_train_np = X_train.numpy()
print(X_train_np)

In [ ]:
# Lade Bibliothek zum Kippen des Bildes
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip

# Führe Transformation aus
X_train = RandomFlip(mode='horizontal')(X_train)
X_test = RandomFlip(mode='horizontal')(X_test)

# NUR zum TESTEN! -> unplausible Transformation
#X_train = RandomFlip(mode='vertical')(X_train) 

In [ ]:
# Gebe ein beliebiges Bild aus
rnd_image = np.random.randint(0, X_train.shape[0]) # Wähle zufällig einen Index

plt.imshow(X_train[rnd_image])
plt.xticks([])
plt.yticks([])
plt.xlabel("Bild #%d (%s)\n" % (rnd_image, labels[y_train[rnd_image][0]]))

In [ ]:
# Dimensionen der Trainingsdaten
print(X_train.shape)

# Sicherstellen, dass Trainings und Testdaten die gleiche Dimension haben
assert X_train.shape[1:] == X_test.shape[1:] # Ignoriere Batch Dimension

## Lade Basis-Architektur

Verwende die MobileNet Architektur: https://arxiv.org/abs/1704.04861

In [ ]:
# Lade Bibliotheken
from tensorflow.keras.applications import MobileNet # https://www.tensorflow.org/api_docs/python/tf/keras/applications/MobileNet

In [ ]:
# Lade das Modell und die gelernten Gewichte für den Datensatz "Imagenet"
base_model = MobileNet(input_shape=(None, None, 3), include_top=False, weights="imagenet")

In [ ]:
# Verhindere Parameteranpassungen im Base Model
base_model.trainable = False

In [ ]:
# Zeige Base-model Struktur (also die des MobileNet) - ohne DNN Anteil
base_model.summary()

In [ ]:
# Zum Vergleich: mit DNN zur Klassifizierung
base_model_2 = MobileNet(input_shape=(224, 224, 3),include_top=True, weights="imagenet")
base_model_2.summary()

del base_model_2

In [ ]:
# Lade Bibliotheken

import tensorflow as tf

# Input Layer
from tensorflow.keras import Input, Model

# Layer für Klassifikator
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout

In [ ]:
# Prüfe wieviele Label/Klassen in der Aufgabe vorkommen 
print(np.unique(y_train))

# Bestimme die Anzahl der Knoten im Ausgabelayer
num_classes = len(np.unique(y_train))
print(num_classes)

In [ ]:
# Input Layer
input = Input(shape=X_train.shape[1:], name='Input')


# BLOCK: Datenvorberietung für Modell
#############################
x = tf.keras.applications.mobilenet.preprocess_input(input)

# BLOCK: Merkmalsextraktion
#############################

x = base_model(x, training = False)

# BLOCK: TRANSFORMATION
#############################

# Tranformation 2D -> 1D
x = Flatten(name='FLATTEN')(x)

# BLOCK: Klassifikation
#############################

# FC Layer 1
x = Dense(512, name='Hidden1')(x)
x = Activation('relu', name='Hidden1_Act')(x)
x = Dropout(0.2)(x)

# FC Layer 2
x = Dense(128, name='Hidden2')(x)
x = Activation('relu', name='Hidden2_Act')(x)
x = Dropout(0.2)(x)

# FC Layer 3
x = Dense(64, name='Hidden3')(x)
x = Activation('relu', name='Hidden3_Act')(x)
x = Dropout(0.2)(x)

# FC Layer 4
x = Dense(32, name='Hidden4')(x)
x = Activation('relu', name='Hidden4_Act')(x)
x = Dropout(0.2)(x)

# Output Layer
x = Dense(num_classes, name='Output')(x)
output = Activation('softmax', name='Output_softmax')(x)

model = Model(input, output)

In [ ]:
# Zeige Modellstruktur
model.summary()

In [ ]:
# Modell als Bild ausgeben
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='MobileNet_base.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Modell zusammenbauen
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001))

## Vorbereiten der Label

In [ ]:
# Lade Bibliothek
from tensorflow.keras.utils import to_categorical

# Prüfe Labeldimension
print(y_train.shape) # 50000x1 Vektor

# Umwandlung in One-Hot Vektor (vgl. Dokumentation "to_categorical")
y_train_1hot = to_categorical(y_train, num_classes)
y_test_1hot = to_categorical(y_test, num_classes)

## Model trainieren

Erinnerung: Nur der "Kopf" Bereich des Modells wird trainiert - das Basismodell ist eingefroren.

In [ ]:
# Trainieren und Verlauf des Trainings speichern
history = model.fit(X_train, y_train_1hot, validation_data=(X_test, y_test_1hot), batch_size=200, epochs=5, verbose=1)

## Trainingsverlauf analysieren

In [ ]:
# Verläufe zeichnen
plt.figure(figsize=(15,9))
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoche')
plt.ylabel('Accuracy')
plt.legend()

## Fein-Tuning

In [ ]:
# Modell "analysieren"
print("Anzahl Layer in Basis Modell (MobileNet): ", len(base_model.layers))
print("Anzahl Layer in zusammengesetzen Modell (MobileNet + eigenes DNN): ", len(model.layers)) # Achtung: Die einzelnen MobileNet Layer sind hier in einem Layer abstrahiert!

# Basis Modell 
base_model.trainable = True

# Ab welchem Layer erlauben wir Parameteränderungen?
num_layer_trainable = 65

# Die ersten #<num_layer_trainable> Layer wieder vom Training ausschließen! Hinweis: Sind ja die Basismerkmale, die "feineren" Merkmale - also die tieferen Layer - sollen an aktuellen Datensatz angepasst werden.
print("Die ersten ", num_layer_trainable, " Layer des Basismodells werden wieder eingefroren.")
for layer in base_model.layers[num_layer_trainable:]:
  layer.trainable =  False

In [ ]:
#base_model.layers

In [ ]:
#base_model.layers[:43]

In [ ]:
# Modell zusammenbauen
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001)) # Hinweis: Lernrate sollte nun etwas kleiner sein (wegen Anzahl trainierbarer Layer)

In [ ]:
# Modell anzeigen
model.summary()

In [ ]:
# Weiter-Trainieren und Verlauf des Trainings speichern
# Nochmal 5 Epochen dranhängen (5+5=10)

history_fine_tuning = model.fit(X_train, y_train_1hot, validation_data=(X_test, y_test_1hot), batch_size=200, epochs=10, initial_epoch=history.epoch[-1] ,verbose=1)

250/250 [==============================] - 18s 73ms/step - loss: 2.3085 - accuracy: 0.1013 - val_loss: 2.3032 - val_accuracy: 0.1000
Epoch 7/10
 75/250 [========>.....................] - ETA: 11s - loss: 2.3062 - accuracy: 0.1007

## Model verifizieren

In [ ]:
# Model verifizieren auf Testdaten
model.evaluate(X_test, y_test_1hot, verbose=1)

## Weiteren Trainingsverlauf analysieren

In [ ]:
# Verläufe zeichnen
plt.figure(figsize=(15,9))
plt.plot(history_fine_tuning.history['accuracy'], label='accuracy')
plt.plot(history_fine_tuning.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoche')
plt.ylabel('Accuracy')
plt.legend()